# Part 1: Getting the initial prompt

In [1]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "[YOUR_KEY_HERE]"
openai.api_key = "[YOUR_KEY_HERE]"

In [23]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

good_examples = [
    """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer."""]
bad_examples = [
    "Below is a question from our forum. You have access to the relevant parts of our documentation to answer it. If you don't know the answer, please say so clearly. False information in this context is much worse than no information at all. [USER_QUESTION_HERE]",
    "Please explain. [USER_QUESTION_HERE]"
]


llm = OpenAI(temperature=0.7, model_name="gpt-3.5-turbo")
template = """
I'm looking for the optimal initial prompt for a documentation QA chatbot.
The prompt should instruct the documentation bot on how to best answer the question.

Below are examples of both good prompts and bad prompts as evaluated by a human annotator.
Your job is to generate a new unique instruction prompt that you think will be evaluated better than the example prompts.
You're essentially interpolating in embeddingspace in search of the optimal prompt.
I want you to create an embeddingspace of the examples and create something in the middle.

Only answer with the exact prompt you generated, no extra text is to be added.
The special token [USER_QUESTION_HERE] is used to denote where the actual user question would be inserted.

Examples of GOOD prompts:
{good_examples}

Examples of BAD prompts:
{bad_examples}

Optimal Prompt:
"""

prompt = PromptTemplate(
    input_variables=["good_examples", "bad_examples"],
    template=template,
)

chain = LLMChain(llm=llm, prompt=prompt)


/home/victor/Projects/langchain/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/Projects/langchain/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Restart here

In [25]:
instructions = chain.run(good_examples="\n\n".join(good_examples), bad_examples="\n\n".join(bad_examples))
print(instructions)

Using the information provided in our documentation, craft a clear and accurate answer to [USER_QUESTION_HERE]. If you are unsure of the answer, please state so explicitly. Always include a "SOURCES" section in your response.


# TSNE: where do the new instructions lie?

In [26]:
import pandas as pd
df = pd.DataFrame()
df['text'] = good_examples + bad_examples
df['color'] = ['darkgreen'] * len(good_examples) + ['red'] * len(bad_examples)
df['label'] = ['good'] * len(good_examples) + ['bad'] * len(bad_examples)
df

,text,color,label
0,Given the following extracted parts of a long ...,darkgreen,good
1,Below is a question from our forum. You have a...,red,bad
2,Please explain. [USER_QUESTION_HERE],red,bad
3,"Based on the user's question, provide a concis...",red,bad


In [9]:
import pandas as pd

test = [{'col1': 1, 'col2': 2}, {'col1': 3, 'col2': 4}]
df = pd.DataFrame(test)

df = df.append({'col1': 0, 'col2': 0}, ignore_index=True)
df

/tmp/ipykernel_7890/1642242154.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'col1': 0, 'col2': 0}, ignore_index=True)


TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [27]:
# Embedding the examples
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
df['embedding'] = df['text'].apply(embeddings.embed_query)
df


,text,color,label,embedding
0,Given the following extracted parts of a long ...,darkgreen,good,"[-0.0026994748041033745, 0.020883996039628983,..."
1,Below is a question from our forum. You have a...,red,bad,"[-0.00016475838492624462, 0.028225881978869438..."
2,Please explain. [USER_QUESTION_HERE],red,bad,"[-0.001994671067222953, -0.009613985195755959,..."
3,"Based on the user's question, provide a concis...",red,bad,"[0.004512029234319925, 0.01682089827954769, -0..."


In [28]:
# Embedding the newly incoming one
newly_generated_sample = {'text': instructions, 'color': 'orange', 'label': 'unknown', 'embedding': embeddings.embed_query(instructions)}
# create a new dataframe with the new row
new_df = pd.DataFrame([newly_generated_sample])

# append the new dataframe to the original dataframe
df = df.append(new_df, ignore_index=True)

df

/tmp/ipykernel_36933/32911021.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)


,text,color,label,embedding
0,Given the following extracted parts of a long ...,darkgreen,good,"[-0.0026994748041033745, 0.020883996039628983,..."
1,Below is a question from our forum. You have a...,red,bad,"[-0.00016475838492624462, 0.028225881978869438..."
2,Please explain. [USER_QUESTION_HERE],red,bad,"[-0.001994671067222953, -0.009613985195755959,..."
3,"Based on the user's question, provide a concis...",red,bad,"[0.004512029234319925, 0.01682089827954769, -0..."
4,Using the information provided in our document...,orange,unknown,"[0.01666226051747799, 0.02018873393535614, -0...."


In [10]:
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# Create a t-SNE model and transform the data
tsne = TSNE(n_components=2, perplexity=2, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(np.array(df['embedding'].to_list()))

x = [x for x,y in vis_dims]
y = [y for x,y in vis_dims]
colors = df['color'].to_list()

plt.scatter(x, y, c=colors)
plt.title("Instructions generated by OpenAI using t-SNE")

KeyError: 'embedding'

# Part 2: Using the new prompt to answer questions from our documentation

In [34]:
import faiss
import pickle

from langchain import VectorDBQAWithSourcesChain
from langchain.chat_models import ChatOpenAI


# Load the LangChain.
index = faiss.read_index("docs.index")

with open("faiss_store.pkl", "rb") as f:
    store = pickle.load(f)

# Get the original chainlang prompt and add our own instructions
combine_prompt_template = """{instructions}

Examples

QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: 28-pl

QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: 0-pl
Content: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: 24-pl
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
Source: 5-pl
Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
Source: 34-pl
=========
FINAL ANSWER: The president did not mention Michael Jackson.
SOURCES:

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question", "instructions"]
)

store.index = index
chain = VectorDBQAWithSourcesChain.from_llm(llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2), vectorstore=store, combine_prompt=combine_prompt)
print(chain)

memory=None callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcc9a40c7f0> verbose=False combine_documents_chain=MapReduceDocumentsChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcc9a40c7f0>, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcc9a40c7f0>, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True), llm=ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcc9a40c7f0>, client=<class 'opena

/home/victor/Projects/langchain/.venv/lib/python3.10/site-packages/langchain/chains/qa_with_sources/vector_db.py:57: UserWarning: `VectorDBQAWithSourcesChain` is deprecated - please use `from langchain.chains import RetrievalQAWithSourcesChain`
  warnings.warn(


In [35]:
# Now ask it several questions so it can be scored
questions = [
    "How do I create ClearML credentials?",
    "How does clearml ochestration and scheduling work?",
    "Can you clone an entire ClearML Project?",
    "How do I go to the built-in ClearML Serving dashboard?",
    "Where can I find the datalake in ClearML?"
]

In [37]:
from colored import fg, attr

instructions = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer."""

for question in questions:
    reply = chain({"question": question, "instructions": instructions})
    print(f'{fg("green")}{attr("bold")}{reply["question"]}{attr("reset")}\n\n{fg("red")}{reply["answer"]}\n\n')


How do I create ClearML credentials?

To create ClearML credentials, go to the desired workspace's panel in the ClearML Web UI, click on "App Credentials", and then click on "+ Create new credentials". A dialog will pop up where you can input a label for the new credentials. The dialog displays new credentials, formatted as a ready-to-copy configuration file section (including server configuration information). There are different ways to access the "Create new credentials" option, depending on the ClearML documentation you are following. 





KeyboardInterrupt: 

In [24]:
bad_examples.append(instructions)

In [ ]:
good_examples.append(instructions)